In [1]:
import random
import json
import pandas as pd
import os
import json

In [12]:
csv_file_path = "/Users/meetraval/Desktop/data/Legal/Legal_Final_Data.csv"

model_output_path = "/Users/meetraval/Desktop/data/Legal/deepseek"


In [13]:
df = pd.read_csv(csv_file_path)

In [14]:
# Path to the folder containing the .txt files

# Create a list to store the contents in order of df index
new_column_data = []

for idx in df.index:
    file_path = os.path.join(model_output_path, f"{idx}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            content_lst = f.read().strip().split("```json")
            if len(content_lst) > 1:
              content = json.loads(content_lst[1].split("```")[0].replace("\n", ""))
            else:
              try:
                content = json.loads(content_lst[0])
              except Exception as e:
                content_1 = """{\n  "topics":""" + content_lst[0].split("""{\n  "topics":""")[1]
                content = json.loads(content_1.replace("\n", ""))
            new_column_data.append(content)
    else:
        print(idx)
        new_column_data.append(None)  # or '' or np.nan



In [15]:
print(len(new_column_data))
print(len(df['Conversation'].to_list()))


2166
2166


In [16]:
# Add as a new column
df['predicted_response'] = new_column_data

In [17]:
import ast
# Clean json responses
topics_pred, topics_true, sents_pred, sents_true = [], [], [], []
for ii in range(len(df)):
    json_true = ast.literal_eval(df['Filename_Json'][ii])
    json_pred = df['predicted_response'][ii]
    topic_true = list(json_true.keys())
    sent_true = list(json_true.values())
    topic_pred = [x['topic'] for x in json_pred['topics']]
    sent_pred = [x['sentiment'] for x in json_pred['topics']]
    topics_pred.append(topic_pred)
    topics_true.append(topic_true)
    sents_pred.append(sent_pred)
    sents_true.append(sent_true)

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix

# Step 1: Binarize
mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(topics_true)
y_pred_bin = mlb.transform(topics_pred)

# Step 2: Compute confusion matrices
conf_matrices = multilabel_confusion_matrix(y_true_bin, y_pred_bin)

# Step 3: Print with labels
for label, matrix in zip(mlb.classes_, conf_matrices):
    print(f"Label '{label}':\n{matrix}\n")


Label 'Applicable Laws and Regulations':
[[1888   47]
 [  39  192]]

Label 'Client Objectives and Priorities':
[[1850   29]
 [   5  282]]

Label 'Communication with Opposing Parties':
[[1908   10]
 [   5  243]]

Label 'Confidentiality and Privilege':
[[1835    0]
 [   1  330]]

Label 'Contract Terms and Obligations':
[[1861   26]
 [  12  267]]

Label 'Costs, Fees, and Budgeting':
[[1795   16]
 [   4  351]]

Label 'Dispute Resolution Options':
[[1852   40]
 [  11  263]]

Label 'Drafting and Reviewing Legal Documents':
[[1890   44]
 [   5  227]]

Label 'Enforcement and Implementation Issues':
[[1932    4]
 [  16  214]]

Label 'Evidence and Documentation Review':
[[1866   35]
 [   9  256]]

Label 'Governing Law and Jurisdiction':
[[1885    1]
 [  20  260]]

Label 'Legal Argument and Case Theory':
[[1900   14]
 [  15  237]]

Label 'Legal Issue Identification':
[[1873   62]
 [  33  198]]

Label 'Regulatory Compliance Check':
[[1884  108]
 [   0  174]]

Label 'Risk Assessment and Mitigation'

In [19]:
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)

# --- 1. Per-Class & Aggregate Report
print(classification_report(y_true_bin, y_pred_bin, target_names=mlb.classes_))

# --- 2. Subset Accuracy (exact match)
subset_acc = accuracy_score(y_true_bin, y_pred_bin)
print(f"Subset Accuracy: {subset_acc:.4f}")

# --- 3. Micro-Averaged Scores
print("Micro Precision:", precision_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro Recall:", recall_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='micro'))

# --- 4. Macro-Averaged Scores
print("Macro Precision:", precision_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro Recall:", recall_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='macro'))

# --- 5. Weighted-Averaged Scores
print("Weighted Precision:", precision_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted Recall:", recall_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted F1 Score:", f1_score(y_true_bin, y_pred_bin, average='weighted'))

                                        precision    recall  f1-score   support

       Applicable Laws and Regulations       0.80      0.83      0.82       231
      Client Objectives and Priorities       0.91      0.98      0.94       287
   Communication with Opposing Parties       0.96      0.98      0.97       248
         Confidentiality and Privilege       1.00      1.00      1.00       331
        Contract Terms and Obligations       0.91      0.96      0.93       279
            Costs, Fees, and Budgeting       0.96      0.99      0.97       355
            Dispute Resolution Options       0.87      0.96      0.91       274
Drafting and Reviewing Legal Documents       0.84      0.98      0.90       232
 Enforcement and Implementation Issues       0.98      0.93      0.96       230
     Evidence and Documentation Review       0.88      0.97      0.92       265
        Governing Law and Jurisdiction       1.00      0.93      0.96       280
        Legal Argument and Case Theory 

In [20]:
# Step 1: Binarize
mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(sents_true)
y_pred_bin = mlb.transform(sents_pred)

# Step 2: Compute confusion matrices
conf_matrices = multilabel_confusion_matrix(y_true_bin, y_pred_bin)

# Step 3: Print with labels
for label, matrix in zip(mlb.classes_, conf_matrices):
    print(f"Label '{label}':\n{matrix}\n")

Label 'negative':
[[ 535    3]
 [ 215 1413]]

Label 'neutral':
[[ 495  154]
 [ 484 1033]]

Label 'positive':
[[ 222  281]
 [   3 1660]]



In [21]:
# --- 1. Per-Class & Aggregate Report
print(classification_report(y_true_bin, y_pred_bin, target_names=mlb.classes_))

# --- 2. Subset Accuracy (exact match)
subset_acc = accuracy_score(y_true_bin, y_pred_bin)
print(f"Subset Accuracy: {subset_acc:.4f}")

# --- 3. Micro-Averaged Scores
print("Micro Precision:", precision_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro Recall:", recall_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='micro'))

# --- 4. Macro-Averaged Scores
print("Macro Precision:", precision_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro Recall:", recall_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='macro'))

# --- 5. Weighted-Averaged Scores
print("Weighted Precision:", precision_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted Recall:", recall_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted F1 Score:", f1_score(y_true_bin, y_pred_bin, average='weighted'))

              precision    recall  f1-score   support

    negative       1.00      0.87      0.93      1628
     neutral       0.87      0.68      0.76      1517
    positive       0.86      1.00      0.92      1663

   micro avg       0.90      0.85      0.88      4808
   macro avg       0.91      0.85      0.87      4808
weighted avg       0.91      0.85      0.87      4808
 samples avg       0.91      0.85      0.87      4808

Subset Accuracy: 0.5836
Micro Precision: 0.9036091549295775
Micro Recall: 0.8539933444259568
Micro F1 Score: 0.8781009409751925
Macro Precision: 0.9077905939311126
Macro Recall: 0.849027130376587
Macro F1 Score: 0.8712118760782778
Weighted Precision: 0.9082743959912345
Weighted Recall: 0.8539933444259568
Weighted F1 Score: 0.8740496765617373
